In [1]:
import polars as pl
from polars import col as c
import networkx as nx
from networkx_function import generate_nx_edge
from typing_extensions import Union
import numpy as np
import scipy as sp
from distflow_algorithm import DistFlow, generate_connections_matrix, distflow_algorithm
import time 
import pandapower as pp
import pandapower.networks as pn
from polars_function import get_transfo_admittance, get_transfo_impedance, get_transfo_conductance, get_transfo_imaginary_component

from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb

from copy import deepcopy

In [2]:
# ext_grid_id = "1" 
# n = 0.95
# v_ext_grid_sq = 1.05

# line_data: pl.DataFrame = pl.DataFrame({
#     "line_id":np.arange(1, 13).astype(str),
#     "type": ["branch"]*10 + ["transformer"]*2,
#     "u_of_edge": ["2", "2", "2", "4", "5",  "6", "7", "10", "12", "13", "8", "9"],
#     "v_of_edge": ["11", "1", "3", "1", "4", "4", "8",  "9", "11", "11", "4", "8"],
#     "n_transfo": [1.0]* 10 + [n**2, n**2],
#     "x_pu": np.arange(1, 13)*2e-3,
#     "r_pu": np.arange(1, 13)*1e-3,
#     "b_pu": list(np.arange(1, 11)*- 1e-3) + [0.001, 0.001],
#     "g_pu": [0.]* 10 + [0.001, 0.001]
# })

# node_data: pl.DataFrame = pl.DataFrame({
#     "node_id":np.arange(1, 14).astype(str),
#     "v_base": [400.0]*7 + [200.0, 400.0  , 400.0  , 200.0 , 100.0, 100.0],
#     "p_node_pu": np.array([0, 10, 2, 0, 0, -2, 0, 7, 25, 0, 100, 0.5, 10])*1e-2,
#     "q_node_pu": np.array([0, 0, 0.2, 2, 0, -0.2, 0, 0.7, 0, 0, 0, 0.5, 1])*1e-2,
#     }, strict=False)

# np.set_printoptions(linewidth=200)
# distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
# node_data = pl.DataFrame({"node_id": distflow.node_list}).join(node_data, on="node_id", how="left")
# s_node = node_data["p_node_pu"].to_numpy() + 1j*node_data["q_node_pu"].to_numpy()
# tic = time.time()
# p_flow, q_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v_ext_grid_sq)
# print("f:", time.time() - tic)
# print(distflow.idx)

In [2]:
s_base = 1e6
net: pp.pandapowerNet = pp.from_pickle("../data/input_grid/modified_cigre_network_lv.p")
pp.runpp(net)

ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0]**2
node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

h 0.04647707939147949


In [ ]:
distflow

In [7]:
self = distflow

def get_edge_data(nx_graph: nx.Graph) -> pl.DataFrame:
    
    return pl.DataFrame(
        list(nx_graph.edges(data=True)), 
        strict=False, orient="row", 
        schema=["u_of_edge", "v_of_edge", "data"]
    ).unnest("data")

In [12]:
self.node_parameters

node_id,n_transfo,r_pu,x_pu,g_tra_pu,b_tra_pu,name,type
i64,f64,f64,f64,f64,f64,str,str
26,1.0,0.0,0.0,0.0,0.0,null,null
27,1.0,0.000014,0.000007,0.0,-1.1133e-9,"""Line R1-R2""","""branch"""
14,1.0,0.0,0.0,0.0,0.004039,"""S1""","""switch"""
42,1.0,0.0,0.0,0.0,0.000921,"""S2""","""switch"""
36,1.0,0.0,0.0,0.0,0.002561,"""S3""","""switch"""
…,…,…,…,…,…,…,…
41,1.0,0.378131,0.062681,0.0,-0.00003,"""Line C8-C19""","""branch"""
24,1.0,0.035438,0.0182,0.0,-0.000003,"""Line R9-R10""","""branch"""
23,1.0,0.154125,0.015881,0.0,-0.000012,"""Line R9-R17""","""branch"""


In [ ]:

node_parameters
pl.DataFrame({"v_of_edge": [self.ext_grid_nb]})

v_of_edge
i64
26


In [ ]:
node_parameters = get_edge_data(nx_graph=self.nx_tree_grid)

node_parameters = node_parameters.with_columns(
    pl.when(c("type")== "branch")
    .then(c("b_pu")/2)
    .otherwise(c("b_pu"))
)

downstream_b: pl.DataFrame = node_parameters.filter(c("type") == "branch").group_by("u_of_edge").agg(c("b_pu").sum().alias("b_tra_pu"))
node_parameters.join(downstream_b, left_on="v_of_edge", right_on="u_of_edge", how="left").select(
    c("v_of_edge").alias("node_id")
)

u_of_edge,v_of_edge,name,r_pu,x_pu,b_pu,type,g_pu,n_transfo,b_tra_pu
i64,i64,str,f64,f64,f64,str,f64,f64,f64
26,27,"""Line R1-R2""",0.000014,0.000007,-1.1133e-9,"""branch""",0.0,1.0,null
27,14,"""S1""",0.0,0.0,0.0,"""switch""",0.0,1.0,null
27,42,"""S2""",0.0,0.0,0.0,"""switch""",0.0,1.0,null
27,36,"""S3""",0.0,0.0,0.0,"""switch""",0.0,1.0,null
14,11,"""Trafo R0-R1""",0.02,0.08,0.008078,"""transformer""",0.005,1.0,-0.000003
…,…,…,…,…,…,…,…,…,…
7,41,"""Line C8-C19""",0.378131,0.062681,-0.00003,"""branch""",0.0,1.0,null
5,24,"""Line R9-R10""",0.035438,0.0182,-0.000003,"""branch""",0.0,1.0,-0.000012
5,23,"""Line R9-R17""",0.154125,0.015881,-0.000012,"""branch""",0.0,1.0,null


In [6]:
self.line_data

u_of_edge,v_of_edge,name,r_pu,x_pu,b_pu,type,g_pu,n_transfo
i64,i64,str,f64,f64,f64,str,f64,f64
11,31,"""Line R1-R2""",0.035438,0.0182,-0.000006,"""branch""",0.0,1.0
31,29,"""Line R2-R3""",0.035438,0.0182,-0.000006,"""branch""",0.0,1.0
29,38,"""Line R3-R4""",0.035438,0.0182,-0.000006,"""branch""",0.0,1.0
38,4,"""Line R4-R5""",0.035438,0.0182,-0.000006,"""branch""",0.0,1.0
4,30,"""Line R5-R6""",0.035438,0.0182,-0.000006,"""branch""",0.0,1.0
…,…,…,…,…,…,…,…,…
42,6,"""Trafo I0-I1""",0.07373,0.29423,0.001842,"""transformer""",0.001814,1.080776
36,22,"""Trafo C0-C1""",0.033125,0.133125,0.005122,"""transformer""",0.0025,1.020304
27,14,"""S1""",0.0,0.0,0.0,"""switch""",0.0,1.0


In [ ]:

distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
v0_sq = distflow.v_in_sq*v_ext_grid_sq
node_data = pl.DataFrame({"node_id": distflow.node_list}).join(node_data, on="node_id", how="left")
s_node = node_data["p_node_pu"].to_numpy() + 1j*node_data["q_node_pu"].to_numpy()
p_flow, q_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq)

line_res_pp = pl.DataFrame({
    "node_id": list(net.line["to_bus"]),
    "p_pp": list(net.res_line["p_from_mw"]),
    "q_pp": list(net.res_line["q_from_mvar"])
})

result: pl.DataFrame = node_data.select(
    c("node_id").cast(pl.Int32),
    pl.Series(v).alias("v"),
    pl.Series(p_flow).alias("p"),
    pl.Series(q_flow).alias("q"),
).sort("node_id").with_columns(
    pl.Series(list(net.res_bus["vm_pu"])).alias("v_pp")
).join(line_res_pp, on="node_id", how="left").with_columns(
    (c("v") - c("v_pp")).abs().alias("diff_v"),
    (c("p") - c("p_pp")).abs().alias("diff_p"),
    (c("q") - c("q_pp")).abs().alias("diff_q")
)
print(result.max())


In [18]:
[sorted(generation) for generation in nx.topological_generations(distflow.nx_tree_grid)]

[['44'],
 ['0'],
 ['1', '20', '23'],
 ['2', '21', '24'],
 ['22', '25', '3'],
 ['26', '4'],
 ['12', '27', '33', '5'],
 ['13', '28', '34', '37', '6'],
 ['14', '29', '35', '36', '38', '7'],
 ['15', '17', '30', '39', '40', '8'],
 ['16', '31', '41', '9'],
 ['10', '32', '42'],
 ['11', '18', '43'],
 ['19']]

In [37]:

net: pp.pandapowerNet = pp.from_pickle("../data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2
s_base = 1e6
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0]**2

node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
node_data = pl.DataFrame({"node_id": distflow.node_list}).join(node_data, on="node_id", how="left")
s_node = node_data["p_node_pu"].to_numpy() + 1j*node_data["q_node_pu"].to_numpy()


h 3.3954458236694336


In [22]:
len(list(nx.topological_generations(distflow.nx_tree_grid)))

25

In [5]:
dict_data = {"node_data": node_data, "line_data": line_data}

dict_to_duckdb(dict_data, file_path="../data/input_grid/synthesized_grid.duckdb")

Save dictionary into duckdb file:   0%|                                                                                         | 0/2 [00:00<?, ?it/s]

Save dictionary into duckdb file: 100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 69.93it/s]


In [89]:
node_mapping = dict(zip(distflow.node_list, range(len(distflow.node_list))))
succesor = list(zip(*nx.all_pairs_shortest_path_length(distflow.nx_tree_grid)))
keys_lists = list(map(lambda d: list(d.keys()), succesor[1]))

succesor_pair = pl.DataFrame({"x": succesor[0], "y": keys_lists}).explode("y").with_columns(
    c(col).replace_strict(node_mapping, default=None) for col in ["x", "y"]
)


h_np= np.zeros([len(distflow.node_list), len(distflow.node_list)])
h_np[succesor_pair["x"].to_list(), succesor_pair["y"].to_list()] = 1


In [90]:
h_np

array([[1., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [85]:
h_np

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [73]:
len(distflow.node_list)* len(distflow.node_list)

31292836

In [76]:
succesor_pair

[[0, 0],
 [0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 9],
 [0, 10],
 [0, 11],
 [0, 12],
 [0, 13],
 [0, 14],
 [0, 15],
 [0, 16],
 [0, 17],
 [0, 18],
 [0, 19],
 [0, 20],
 [0, 21],
 [0, 22],
 [0, 23],
 [0, 24],
 [0, 25],
 [0, 26],
 [0, 27],
 [0, 28],
 [0, 29],
 [0, 30],
 [0, 31],
 [0, 32],
 [0, 33],
 [0, 34],
 [0, 35],
 [0, 36],
 [0, 37],
 [0, 38],
 [0, 39],
 [0, 40],
 [0, 41],
 [0, 42],
 [0, 43],
 [0, 44],
 [0, 45],
 [0, 46],
 [0, 47],
 [0, 48],
 [0, 49],
 [0, 50],
 [0, 51],
 [0, 52],
 [0, 53],
 [0, 54],
 [0, 55],
 [0, 56],
 [0, 57],
 [0, 58],
 [0, 59],
 [0, 60],
 [0, 61],
 [0, 62],
 [0, 63],
 [0, 64],
 [0, 65],
 [0, 66],
 [0, 67],
 [0, 68],
 [0, 69],
 [0, 70],
 [0, 71],
 [0, 72],
 [0, 73],
 [0, 74],
 [0, 75],
 [0, 76],
 [0, 77],
 [0, 78],
 [0, 79],
 [0, 80],
 [0, 81],
 [0, 82],
 [0, 83],
 [0, 84],
 [0, 85],
 [0, 86],
 [0, 87],
 [0, 88],
 [0, 89],
 [0, 90],
 [0, 91],
 [0, 92],
 [0, 93],
 [0, 94],
 [0, 95],
 [0, 96],
 [0, 97],
 [0, 98],
 [0, 99],
 [0, 100],

x,y
str,str
"""5593""","""5593"""
"""5593""","""5377"""
"""5593""","""5473"""
"""5593""","""1453"""
"""5593""","""5588"""
…,…
"""5477""","""5477"""
"""669""","""669"""
"""1704""","""1704"""


In [36]:
h_pl = pl.from_dicts(succesor[1])
h_pl = h_pl * 0 + 1

In [7]:
tic = time.time()
p_flow, q_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v_ext_grid_sq)
print("f:", time.time() - tic)
print(distflow.idx)

f: 3.147996664047241
23


In [8]:
tic = time.time()
distflow_algorithm(h_np=distflow.h_np, g_np=distflow.g_np, y=distflow.y, z= distflow.z, dv_prop_np=distflow.dv_prop_np, s_node=s_node, v0_sq=v_ext_grid_sq)
print(distflow.idx)

UnboundLocalError: cannot access local variable 'numpy_copy' where it is not associated with a value

In [ ]:
s_node

array([0.     +0.j, 0.     +0.j, 0.     +0.j, ..., 0.00276+0.j,
       0.00182+0.j, 0.00188+0.j])

In [ ]:
v_sq = np.multiply(v_ext_grid_sq, distflow.v_in_sq)
s_init: np.array = np.matmul(distflow.h_np, s_node + v_sq*distflow.y)
i_init: np.array = np.absolute(s_init)**2/v_sq
s_up = s_init + np.matmul(distflow.z_init_np, i_init)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5594 is different from 0)

In [ ]:
s_up

array([2.38690251e+01+5.61098830e+00j, 2.38049811e+01+3.53623406e+00j,
       1.10192510e+00+6.00706090e-02j, ...,
       2.76000000e-03-1.65836474e-04j, 1.82000000e-03-4.97156903e-06j,
       1.88000000e-03-7.41117754e-06j])

In [ ]:
s_init: np.array = np.matmul(distflow.h_np, s_node + v_sq*distflow.y )
s_init

array([2.22713787e+01+1.88558322e+00j, 2.22713787e+01+1.57424715e+00j,
       1.03640196e+00+2.25030189e-02j, ...,
       2.76000000e-03-1.65836474e-04j, 1.82000000e-03-4.97156903e-06j,
       1.88000000e-03-7.41117754e-06j])

In [ ]:
s_init

array([2.22713787e+01+1.88558322e+00j, 2.22713787e+01+1.57424715e+00j,
       1.03640196e+00+2.25030189e-02j, ...,
       2.76000000e-03-1.65836474e-04j, 1.82000000e-03-4.97156903e-06j,
       1.88000000e-03-7.41117754e-06j])

In [ ]:
np.matmul(distflow.h_np, s_node)

array([2.22135841e+01+0.j, 2.22135841e+01+0.j, 1.03548393e+00+0.j, ...,
       2.76000000e-03+0.j, 1.82000000e-03+0.j, 1.88000000e-03+0.j])

In [ ]:
s_up

array([2.38690251e+01+5.61098830e+00j, 2.38049811e+01+3.53623406e+00j,
       1.10192510e+00+6.00706090e-02j, ...,
       2.76000000e-03-1.65836474e-04j, 1.82000000e-03-4.97156903e-06j,
       1.88000000e-03-7.41117754e-06j])

In [ ]:
# tic = time.time()
# pp.runpp(net)
# print("f:", time.time() - tic)

In [ ]:
# s_base = 1e8
# ext_grid_id: str = str(net.ext_grid["bus"][0])
# v_ext_grid_sq: float = net.ext_grid["vm_pu"][0]**2

# node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)


In [ ]:
# distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

# v0_sq = distflow.v_in_sq*v_ext_grid_sq


In [ ]:
# tic = time.time()
# p_flow, q_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq)
# print("f:", time.time() - tic)